# Sztuczna inteligencja i inżynieria wiedzy Lista 1 Raport – Aleksander Stepaniuk 272644

Najważniejsze fragmenty kodu:

- Funkcje pomocnicze (obliczanie odległości, konwersje czasu, wyświetlanie rozkładu jazdy)
- Logger do wyświetlania komunikatów
- Graf połączeń komunikacji miejskiej (klasa `TransitGraph`) oraz połączenia (klasa `Connection`)
- Wyszukiwanie ścieżki za pomocą A* (klasa `PathFinder`)
- Rozwiązanie TSP metodą Tabu Search (klasa `TabuSearchTSP`)


## 1. Reprezentacja grafu
### a) – `Connection`
zawiera informacje o pojedynczym połączeniu między dwoma przystankami, w tym:
- identyfikator,
- przewoźnik,
- numer linii,
- czas odjazdu i przyjazdu,
- nazwy przystanków,
- współrzędne geograficzne.


In [ ]:
def time_to_seconds(time_str: str) -> int:
    pass

class Connection:
    def __init__(self, row):
        self.id = row['id']
        self.company = row['company']
        self.line = row['line']
        self.departure_time = time_to_seconds(row['departure_time'])
        self.arrival_time = time_to_seconds(row['arrival_time'])
        self.start_stop = row['start_stop'].lower()
        self.end_stop = row['end_stop'].lower()
        self.start_stop_lat = float(row['start_stop_lat'])
        self.start_stop_lon = float(row['start_stop_lon'])
        self.end_stop_lat = float(row['end_stop_lat'])
        self.end_stop_lon = float(row['end_stop_lon'])

### b) – `TransitGraph`
klasa zawiera pola:
- `connections_by_stop`: lista połączeń z danego przystanku,
- `edges_by_pair`: lista połączeń między parami przystanków, 
- `nodes`: zbiór sąsiadów, czyli przystanków, do których można dojechać bezpośrednio z danego przystanku,
- `stops_coords`: współrzędne geograficzne przystanków.

In [ ]:
from collections import defaultdict
class TransitGraph:
    def __init__(self, csv_path: str, logger):
        self.logger = logger
        self.connections_by_stop = defaultdict(list)
        self.edges_by_pair = defaultdict(list)
        self.nodes = defaultdict(set)
        self.stops_coords = {}
        self._load_csv(csv_path)

## 2. Wyszukiwanie ścieżki – klasa `PathFinder` (A*)

metoda `optimized_a_star` realizuje wyszukiwanie ścieżki:

- **Heurystyka**: obliczana przez `heuristic_time` na podstawie odległości (funkcja `haversine`) i średniej prędkości.
- **Sąsiedzi**: funkcja `_get_next_connections` wybiera kolejne dostępne połączenia. Zależnie od trybu są to albo wszystkie połączenia po obecnym czasie, albo jedno najlepsze z danego startowego punktu do danego końcowego.




In [ ]:
class PathFinder:
    def __init__(self, graph: TransitGraph, logger, avg_speed_kmh: float = 20.0, transfer_threshold: int = 300):
        self.graph = graph
        self.logger = logger
        self.avg_speed = avg_speed_kmh / 3600.0  # km/s
        self.transfer_threshold = transfer_threshold

## 3. `TabuSearchTSP`

Klasa `TabuSearchTSP` wykorzystuje heurystykę Tabu Search do optymalizacji kolejności odwiedzania przystanków. Zawiera:

#### Pola:
- `graph`: graf połączeń,
- `logger`: logger,
- `pathfinder`: obiekt do wyszukiwania ścieżek,
- `mode`: tryb kosztu (czas lub przesiadki),
- `cost_cache`: dictionary cache dla obliczonych kosztów.

#### Metody:
- `build_cost_matrix`: buduje macierz kosztów między wszystkimi przystankami przy użyciu A* (funkcja `compute_cost_between`),
- `get_neighbors`: generuje sąsiednie rozwiązania przez zamianę elementów (poza punktem startowym).
- `solve`: iteracyjnie poszukuje lepszego rozwiązania, unikając powtórzeń dzięki liście tabu.




In [ ]:
class TabuSearchTSP:
    def __init__(self, graph: TransitGraph, logger, pathfinder: PathFinder, mode='t'):
        self.graph = graph
        self.logger = logger
        self.pathfinder = pathfinder
        self.mode = mode.lower()
        self.cost_cache = {}

`get_neighbors` zwraca rozwiązania odległe o jedno zamienione miejsce:

In [ ]:
import random 

def get_neighbors(self, solution, sample_size=None):
    neighbors = []
    n = len(solution)
    for i in range(1, n):
        for j in range(i + 1, n):
            new_sol = solution[:]
            new_sol[i], new_sol[j] = new_sol[j], new_sol[i]
            neighbors.append(tuple(new_sol))
    if sample_size is not None and len(neighbors) > sample_size:
        neighbors = random.sample(neighbors, sample_size)
    return neighbors

## 4. Funkcja główna – `main`

Funkcja `main` inicjalizuje logger, graf, oraz obiekty odpowiedzialne za wyszukiwanie ścieżek i rozwiązywanie TabuSearch. W zależności od wejścia użytkownika (czy drugi input zawiera średnik) wybierany jest tryb do zadania 1 lub 2. 

Liczy też łączny czas i wyświetla informacje przy użyciu funkcji `print_schedule`.



## 5. Heurystyka

Funkcja ta dla dwóch przystanków oblicza czas podróży między nimi na podstawie odległości i średniej prędkości. Używam tego jako heurystyki w A*.

In [ ]:
import math 

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def heuristic_time(self, current_stop: str, target_stop: str) -> float:
    if current_stop not in self.graph.stops_coords or target_stop not in self.graph.stops_coords:
        return 0
    lat1, lon1 = self.graph.stops_coords[current_stop]
    lat2, lon2 = self.graph.stops_coords[target_stop]
    distance = haversine(lat1, lon1, lat2, lon2)
    return distance / self.avg_speed

## 6. Funkcje kosztu
Wykorzystałem dwie funkcje kosztu:
- **czas podróży**: łączny czas podróży między przystankami (czas oczekiwania + czas transportu),
- **ilość przesiadek**: liczba przesiadek na trasie (zmiana linii).

In [ ]:
# pseudocode dla kosztu czasu
... # fragment a*
if mode in ('t', 'time'):
    wait = eff_dep - current_time
    travel = eff_arr - eff_dep
    new_g = g + wait + travel
    ...
else:
    # mode is transfers ('p')
...

In [ ]:
# pseudocode dla kosztu przesiadek
if current_line is None:
    additional = 0
else:
    additional = 1 if (next_line != current_line or wait > self.transfer_threshold) else 0
new_g = g + additional
new_f = new_g * L + eff_arr



## 7. Wnioski

a) Algorytm Dijkstry zawsze znajdzie najkrótszą trasę, ale kosztem przeszukania wszystkich możliwych ścieżek (dlatego jest wolniejszy).

b) A* jest bardziej efektywny, ponieważ wykorzystuje heurystykę do ograniczenia przeszukiwania. W przypadku dużych grafów A* może być znacznie szybszy niż Dijkstra. Czasami A* może nie znaleźć optymalnej trasy, ale w praktyce często daje bardzo dobre wyniki. 

c) W przypadku problemu TSP Tabu Search jest bardziej efektywny niż czysty brute-force, ale wymaga dobrego dobrania parametrów (np. długości listy tabu, liczby iteracji itd.) do specyfiki problemu. 

## 8. Napotkane problemy
a) Przy dużych grafach (np. 1000 przystanków) algorytm Dijkstry może być bardzo wolny, dlatego użyłem A* z heurystyką.

b) W przypadku TSP Tabu Search może utknąć w lokalnych minimach, dlatego zastosowałem różne techniki, takie jak losowe przeszukiwanie sąsiedztwa i dynamiczne dostosowywanie długości listy tabu.


## 9. Dodatkowe Uwagi
a) Zastosowałem różne metody dodatkowych optymalizacji algorytmów, takie jak cache'owanie kosztów w Tabu Search, czy wyszukiwanie binarne aby przyspieszyć obliczenia.

In [ ]:
import bisect
def get_connections_from(self, stop: str, current_time: int):
    conns, dep_times = self.connections_by_stop[stop]
    current_day_time = current_time % 86400
    idx = bisect.bisect_left(dep_times, current_day_time)
    # ...ciąg dalszy...
    # for c in conns[idx:]:
    #    ....
    # for c in conns[:idx]:
    #    ....
    

In [ ]:
def compute_cost_between(self, start, end, start_time):
    key = (start, end, self.mode, start_time)
    if key in self.cost_cache:
        return self.cost_cache[key]
    if self.mode in ('t', 'time'):
        _, cost, _ = self.pathfinder.optimized_a_star(start, end, start_time, mode='time')
    elif self.mode in ('p', 'transfers'):
        _, cost, _ = self.pathfinder.optimized_a_star(start, end, start_time, mode='transfers')
    else:
        raise ValueError("Nieznany tryb optymalizacji")
    self.cost_cache[key] = cost
    return cost

b) W kodzie używam loggera do wyświetlania komunikatów, co ułatwia debugowanie i śledzenie postępu.

In [ ]:
import logging
import time

class Logger:
    def __init__(self):
        self.start_time = time.time()
        logging.basicConfig(level=logging.INFO, format="%(message)s")
        self.logger = logging.getLogger("JourneyPlanner")

    def log(self, message: str):
        elapsed = time.time() - self.start_time
        self.logger.info(f"[{elapsed:.2f} s] {message}")

c) Kod posiada dwa różne tryby - szybki i dokładny, który pozwala dostosować czy zależy nam na precyzji czy szybkości obliczeń. Dla większości przypadków wystarczy tryb szybki, ale dla bardziej skomplikowanych tras i dużej ilości czasu (około średnio 100 razy dłużej) można użyć trybu dokładnego.

In [ ]:
# TRUE -> fast, but not always optimal
# FALSE -> slow, but (i think) always optimal
USE_FAST_CONNECTION_SEARCH = True